# Langchain GraphDB

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
import os
import getpass
from langchain_google_genai.llms import GoogleGenerativeAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.graphs import Neo4jGraph


In [27]:
from langchain.schema import Document

loader = TextLoader("output.txt")
documents= loader.load()

full_text = "".join([doc.page_content for doc in documents])

# Take the first 3000 characters
sample_text = full_text[:3000]

sample_doc = [Document(page_content=sample_text)]

text_splitter = CharacterTextSplitter(chunk_size=200 , chunk_overlap= 20)
texts = text_splitter.split_documents(sample_doc)

In [28]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass()
llm = GoogleGenerativeAI(model= "gemini-2.5-flash", temperature= 0.0)


In [29]:
llm_transformer = LLMGraphTransformer(llm= llm)
graph_doc = llm_transformer.convert_to_graph_documents(texts)

In [30]:
for doc in graph_doc[:5]:
    print(doc.dict)

<bound method BaseModel.dict of GraphDocument(nodes=[Node(id='jambon', type='Food', properties={}), Node(id='Grasset', type='Publisher', properties={}), Node(id="HERMÈS II. L'INTERFÉRENCE", type='Book', properties={}), Node(id='Lucrèce', type='Person', properties={}), Node(id='MICHEL SERRES', type='Person', properties={}), Node(id='désordre', type='Concept', properties={}), Node(id='HERMÈS 1. LA COMMUNICATION', type='Book', properties={}), Node(id='LA NAISSANCE DE LA PHYSIQUE DANS LE TEXTE DE LUCRÈCE. FLEUVES ET TURBULENCES', type='Book', properties={}), Node(id='Hermann', type='Publisher', properties={}), Node(id='JOUVENCES. SUR JULES VERNE', type='Book', properties={}), Node(id='Le rat rustique', type='Animal', properties={}), Node(id='Presses Universitaires de France', type='Publisher', properties={}), Node(id='1977', type='Year', properties={}), Node(id='restes de situation', type='Concept', properties={}), Node(id='GELS', type='Book', properties={}), Node(id='distribution', type='

In [31]:
graph_store = Neo4jGraph(url='bolt://localhost:7690' , username= 'neo4j', password= 'administrador')
graph_store.add_graph_documents(graph_doc, baseEntityLabel=True, include_source=True)


In [37]:
from langchain.chains import GraphCypherQAChain

# Optional: inspect schema so the model knows how to query
graph_store.refresh_schema()  # <-- auto-reads node labels and relationships

# Create the QA chain
qa_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph_store,
    allow_dangerous_requests = True,
    cypher_prompt_template="""
You are Michel Serres, a philosopher who must answer using his book and personal opinion

{schema}

Translate the user's question into a Cypher query. Then answer it based on the results.
Question: {question}
""",
    verbose=True
)


In [42]:
question = "What is LE PARASITE"
result = qa_chain.run(question)

print("Answer:", result)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n)
WHERE n.id = 'LE PARASITE'
RETURN n

Full Context:
[{'n': {'id': 'LE PARASITE'}}]

> Finished chain.
Answer: I don't know the answer.


# DEPRECATED

In [1]:
import os
import markdown
from bs4 import BeautifulSoup

def parse_markdown_to_text(md_file_path):
    with open(md_file_path, 'r', encoding='utf-8') as f:
        md_content = f.read()
    html = markdown.markdown(md_content)
    soup = BeautifulSoup(html, 'html.parser')
    return soup.get_text()

def chunk_text(text, chunk_size=500, overlap=100):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i + chunk_size]
        chunks.append(" ".join(chunk))
        i += chunk_size - overlap
    return chunks

# recursive scan of .md files
all_chunks = []
for root, dirs, files in os.walk("notes"):
    for file in files:
        if file.endswith(".md"):
            full_path = os.path.join(root, file)
            text = parse_markdown_to_text(full_path)
            chunks = chunk_text(text)
            for idx, chunk in enumerate(chunks):
                all_chunks.append({
                    "text": chunk,
                    "source": full_path,
                    "chunk_id": idx
                })

# quick preview
print(f"Prepared {len(all_chunks)} chunks")
print(all_chunks[0])


Prepared 96 chunks
{'text': 'Parasite [[Quasi-objet]] [[Thanatocratie]] Biogée Projet 1 Contrat naturel [[Projet 2 Éducation, transdisciplinarité et ouverture]] Hominescence Cosmocratie Connectif Relation [[Structure (mathématiques)]] [[Autosuffisance des mathématiques]] [[Analyse mathématique]] [[Importance du monde sensible en philosophie]] Musique - [[Musique]] - [[Musique et vivant]] - [[Musique et néguentropie]] - [[Musique et relations]] - [[Visitation]] [[Communication]] [[But de la philosophie]] Politique et droit [[Politique du tout]] [[C Concepts et personnages/Concepts/Habiter]]', 'source': 'notes\\B Concepts et personnages\\B Concepts et personnages\\Concepts\\0 Concepts créés par Michel Serres.md', 'chunk_id': 0}


In [2]:
total_words = sum(len(chunk['text'].split()) for chunk in all_chunks)
print(f"Total words across all chunks: {total_words}")


Total words across all chunks: 22901


In [3]:
sources = set(chunk['source'] for chunk in all_chunks)
print(f"Found {len(sources)} unique markdown files:")
for s in sources:
    print(s)


Found 76 unique markdown files:
notes\B Concepts et personnages\B Concepts et personnages\Personnages\Arlequin.md
notes\B Concepts et personnages\B Concepts et personnages\Concepts\0 Concepts créés par Michel Serres.md
notes\B Concepts et personnages\B Concepts et personnages\Mots et idées\Fonction du philosophe.md
notes\B Concepts et personnages\B Concepts et personnages\Mots et idées\Plérôme.md
notes\B Concepts et personnages\B Concepts et personnages\Concepts\Grand récit.md
notes\I Méthodes\I Méthodes\méthode algorithmique.md
notes\B Concepts et personnages\B Concepts et personnages\Concepts\Exodarwinisme.md
notes\B Concepts et personnages\B Concepts et personnages\Mots et idées\But de la philosophie.md
notes\B Concepts et personnages\B Concepts et personnages\Personnages\1.Personnages de Michel Serres (liste SA).md
notes\B Concepts et personnages\B Concepts et personnages\Mots et idées\Humus.md
notes\B Concepts et personnages\B Concepts et personnages\Mots et idées\Paix et divin.md

In [4]:
# choose model
from sentence_transformers import SentenceTransformer

C:\Users\Andre\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embed_model = SentenceTransformer('all-MiniLM-L6-v2')  # swap for distiluse-base-multilingual-cased-v2 if needed

# create embeddings
texts = [chunk['text'] for chunk in all_chunks]
embeddings = embed_model.encode(texts, show_progress_bar=True)

# verify
print(f"Generated embeddings shape: {embeddings.shape}")


Batches: 100%|██████████| 3/3 [00:01<00:00,  1.84it/s]

Generated embeddings shape: (96, 384)


In [6]:
import faiss
import numpy as np

In [7]:
# dimension of the embedding
embedding_dim = embeddings.shape[1]

# build the index
index = faiss.IndexFlatIP(embedding_dim)  # inner product for cosine-like similarity
# normalize if you want cosine:
faiss.normalize_L2(embeddings)

# add embeddings
index.add(embeddings)

# verify
print(f"Indexed {index.ntotal} chunks")


Indexed 96 chunks


In [8]:
faiss.write_index(index, "faiss_index.index")

In [9]:
import json

with open("chunks.json", "w", encoding="utf-8") as f:
    json.dump(all_chunks, f, ensure_ascii=False, indent=2)


In [10]:
# embed the query
query = "What is Michel Serres' theory of communication?"
query_embedding = embed_model.encode([query])
faiss.normalize_L2(query_embedding)

# search
k = 5
D, I = index.search(query_embedding, k)

print("Top results:")
for idx in I[0]:
    chunk = all_chunks[idx]
    print(f"\nSource: {chunk['source']} | chunk {chunk['chunk_id']}")
    print(chunk['text'][:300] + "...")


Top results:

Source: notes\B Concepts et personnages\B Concepts et personnages\Mots et idées\Communication.md | chunk 0
Leçon leibnizienne : la communication optimale n’est pas l’unisson, mais une harmonie saturée de différences ?...

Source: notes\B Concepts et personnages\B Concepts et personnages\Concepts\Point fixe.md | chunk 0
La structure du point fixe est la premiêre grande découverte de Michel Serres....

Source: notes\I Méthodes\I Méthodes\passage du nord-ouest.md | chunk 0
Il existe un passage entre les sciences exactes et les sciences humaines: le passage est rare et resserre […] Des sciences humaines aux sciences exactes, ou inversement, le chemin ne traverse pas un espace homogène et vide. La métaphore de cet archipel extraordinairement compliqué du Grand Nord cana...

Source: notes\I Méthodes\I Méthodes\randonnée.md | chunk 0
Face à l’impossibilité de faire le tour des trois champs du savoir, des hommes et des choses, il faut envisager une randonnée: "chaque discipline, 

In [11]:
# your user query
query = "Explain Michel Serres' concept of communication"

# embed and search (from step 3)
query_embedding = embed_model.encode([query])
faiss.normalize_L2(query_embedding)

k = 5
D, I = index.search(query_embedding, k)

# collect top chunks
top_chunks = [all_chunks[idx]["text"] for idx in I[0]]

# build the context
context = "\n\n".join(top_chunks)

# final prompt
prompt = f"""You are a helpful assistant with expertise in Michel Serres' work.
Use the following context to answer the question.
If you don't know, say so honestly.

Context:
{context}

Question:
{query}

Answer:"""


In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import logging

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-Nemo-Instruct-2407")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-Nemo-Instruct-2407")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:


model_name = "mistralai/Mistral-Nemo-Instruct-2407"  # replace with your actual checkpoint name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=512)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(answer)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]